In [132]:
!pip install recordlinkage
!pip install sklearn


distutils: /usr/local/lib/python3.8/dist-packages
sysconfig: /usr/lib/python3.8/site-packages
distutils: /usr/local/lib/python3.8/dist-packages
sysconfig: /usr/lib/python3.8/site-packages
distutils: /usr/local/include/python3.8/UNKNOWN
sysconfig: /usr/include/python3.8
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/local
sysconfig: /usr
user = False
home = None
root = None
prefix = None
distutils: /home/alessandro/.local/include/python3.8/UNKNOWN
sysconfig: /home/alessandro/.local/include/python3.8
user = True
home = None
root = None
prefix = None
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
distutils: /usr/local/lib/python3.8/dist-packages
sysconfig: /usr/lib/python3.8/site-packages
distutils: /usr/local/lib/python3.8/dist-packages
sysconfig: /usr/lib/python3.8/site-packages
distutils: /usr/local/include/python3.8/UNKNOWN
sysconfig: /usr/include/python3.8
distutils: /usr/local/bin
sysconfig: /usr/bin
distutils: /usr/loca

In [21]:
import numpy as np 
import pandas as pd
import sklearn
import recordlinkage
from sklearn.model_selection import train_test_split

# RECORD LINKAGE AZIENDE

In [22]:
datiAziende = pd.read_csv("/home/alessandro/Scrivania/Homework4/aziendeAlignment.csv")
datiAziende = datiAziende.drop('Unnamed: 0', 1)
datiAziende

,Name,Rank,Sector,CEO,Head Country,website
0,APPLE,1,TECHNOLOGY,TIM COOK,USA,HTTPS://WWW.APPLE.COM/
1,SAUDI ARABIAN OIL COMPANY (SAUDI ARAMCO),3,ENERGY,AMIN H. AL-NASSER,SAUDI ARABIA,HTTPS://WWW.SAUDIARAMCO.COM/
2,AMAZON.COM,5,ECOMMERCE,ANDY JASSY,USA,HTTPS://WWW.AMAZON.COM/
3,MICROSOFT CORPORATION,2,TECHNOLOGY,SATYA NADELLA,USA,HTTPS://WWW.MICROSOFT.COM/
4,ALPHABET,4,TECHNOLOGY,SUNDAR PICHAI,USA,HTTPS://ABC.XYZ/
...,...,...,...,...,...,...
6646,ENTERTAINMENT CUP,NaN,"ENTERTAINMENT,MUSIC",HONG YU JIN,"SEOCHO-GU, SEOUL, SOUTH KOREA",HTTPS://KPOP.FANDOM.COM/WIKI/ENTERTAINMENT_CUP
6647,DR MUSIC,NaN,NaN,NaN,"SEOUL, SOUTH KOREA",HTTPS://KPOP.FANDOM.COM/WIKI/DR_MUSIC
6648,DSP MEDIA,NaN,NaN,LEE HO-YEON,"SEOUL, SOUTH KOREA",HTTPS://KPOP.FANDOM.COM/WIKI/DSP_MEDIA
6649,DREAMUS,NaN,NaN,NaN,"SEOUL, SOUTH KOREA",HTTPS://KPOP.FANDOM.COM/WIKI/DREAMUS


In [23]:
indexer = recordlinkage.Index()
indexer.block('Name')
coppie = indexer.index(datiAziende, datiAziende)

In [24]:
coppie

MultiIndex([(   0,    0),
            (   1,    1),
            (   2,    2),
            (   3,    3),
            (   4,    4),
            (   5,    5),
            (   6,    6),
            (   7,    7),
            (   8,    8),
            (   9,    9),
            ...
            (6641, 6641),
            (6642, 6642),
            (6643, 6643),
            (6644, 6644),
            (6645, 6645),
            (6646, 6646),
            (6647, 6647),
            (6648, 6648),
            (6649, 6649),
            (6650, 6650)],
           length=9681)

In [25]:
print(coppie[523])


(445, 445)


In [26]:
#splitto le coppie 
train_coppie = coppie[:4840]
test_coppie = coppie[4840:]

In [27]:
unique = datiAziende["Name"].unique()
np.size(unique)
#piu di mille nomi duplicati 

5503

In [84]:
compare = recordlinkage.Compare()

compare.string('Name', 'Name', method='jarowinkler', label="Name", threshold=0.80)
compare.string('Sector', 'Sector', method='levenshtein', label="Sector", threshold=0.80)
compare.string('Head Country', 'Head Country', method='levenshtein', label="Head Country", threshold=0.85)
compare.string('website', 'website', method='levenshtein', label="website", threshold=0.85)
compare.string('CEO', 'CEO', method='levenshtein', label="CEO", threshold=0.90)


training_features = compare.compute(train_coppie, datiAziende, datiAziende)
training_features['score'] = training_features.loc[:, 'Name':'CEO'].sum(axis=1)

In [48]:
training_matches = training_features[training_features.sum(axis=1) > 4].reset_index()
training_matches['score'] = training_matches.loc[:, 'Name':'CEO'].sum(axis=1)
toDrop = ['Name', 'Sector', 'Head Country', 'website', 'CEO', 'score']
training_matches = training_matches.drop(toDrop, axis=1)
training_matches.tail()

,level_0,level_1
3144,4192,3274
3145,4192,4192
3146,2516,2516
3147,2516,3923
3148,3923,2516


In [36]:
training_matches = pd.MultiIndex.from_frame(training_matches) #matches conversion

In [51]:
classifier = recordlinkage.KMeansClassifier()
classifier.fit(training_features)

In [52]:
compare = recordlinkage.Compare()

compare.string('Name', 'Name', method='jarowinkler', label="Name", threshold=0.80)
compare.string('Sector', 'Sector', method='levenshtein', label="Sector", threshold=0.80)
compare.string('Head Country', 'Head Country', method='levenshtein', label="Head Country", threshold=0.85)
compare.string('website', 'website', method='levenshtein', label="website", threshold=0.85)
compare.string('CEO', 'CEO', method='levenshtein', label="CEO", threshold=0.90)

test_features = compare.compute(test_coppie, datiAziende, datiAziende)
test_features['score'] = test_features.loc[:, 'Name':'CEO'].sum(axis=1)

In [77]:
test_matches = test_features[test_features.sum(axis=1) > 2].reset_index()
test_matches['score'] = test_matches.loc[:, 'Name':'CEO'].sum(axis=1)
toDrop = ['Name', 'Sector', 'website', 'CEO', 'Head Country','score']
test_matches = test_matches.drop(toDrop, axis=1)
test_matches.head()

,level_0,level_1
0,3923,3923
1,2517,2517
2,2517,3575
3,3575,2517
4,3575,3575


In [78]:
test_matches = pd.MultiIndex.from_frame(test_matches) #matches conversion

In [79]:
predictions = classifier.predict(test_features)

In [83]:
confusion_matrix = recordlinkage.confusion_matrix(test_matches, predictions, len(test_features))

recall = recordlinkage.recall(test_matches, predictions)
print('RECALL -->', recall)
precision = recordlinkage.precision(test_matches, predictions)
print('PRECISION -->', precision)
accuracy = recordlinkage.accuracy(test_matches, predictions, len(test_features))
print('ACCURACY -->', accuracy)

RECALL --> 0.8815306754802726
PRECISION --> 0.91
ACCURACY --> 0.8615306754802726


------------

# RECORD LINKAGE PROFESSIONISTI

In [92]:
datiProf = pd.read_csv("/home/alessandro/Scrivania/Homework4/professionistiAlignment.csv")
datiProf = datiProf.drop('Unnamed: 0', 1)
datiProf

,name,city,sector,specialization,languages,website,tel
0,ANDREA ABATE,ROMA,CIVILE E AMBIENTALE - INDUSTRIALE - DELL'INFOR...,MECCANICA,ITALIANO,NaN,NaN
1,FABIANO ABATE,ROMA,CIVILE E AMBIENTALE - INDUSTRIALE - DELL'INFOR...,INGEGNERIA INFORMATICA E DELL'AUTOMAZIONE,ITALIANO,NaN,NaN
2,GIANVITTORIO ABATE,NAPOLI,CIVILE E AMBIENTALE - INDUSTRIALE - DELL'INFOR...,INGEGNERIA ELETTRONICA,ITALIANO,NaN,NaN
3,LUIGI ABATE,MESSINA,CIVILE E AMBIENTALE - INDUSTRIALE - DELL'INFOR...,INGEGNERIA CIVILE TRASPORTI,ITALIANO,NaN,NaN
4,MASSIMO ABATE,CATANIA,DELL'INFORMAZIONE,INFORMATICA,ITALIANO,NaN,NaN
...,...,...,...,...,...,...,...
27091,STUDIO LEGALE ASSOCIATO CAPOGRECO,NaN,DIRITTO BANCARIO E ASSICURATIVO,NaN,ITALIANO,HTTPS://WWW.PRONTOPROFESSIONISTA.IT/RICERCA-PR...,096722212
27092,GEOMETRA CARLO ALBERTO CONTI,NaN,GEOMETRI,NaN,ITALIANO,HTTPS://WWW.PRONTOPROFESSIONISTA.IT/RICERCA-PR...,0187914204
27093,AVV. BARBARA NINCHERI,NaN,RESPONSABILITÀ CIVILE,NaN,ITALIANO,HTTPS://WWW.PRONTOPROFESSIONISTA.IT/RICERCA-PR...,0553852008
27094,DOTT. ABATE SERGIO,NaN,CONSULENZA FINANZIARIA E PATRIMONIALE,NaN,ITALIANO,HTTPS://WWW.PRONTOPROFESSIONISTA.IT/RICERCA-PR...,3296146031


In [86]:
indexer = recordlinkage.Index()
indexer.block('name')
coppieProf = indexer.index(datiProf, datiProf)

In [87]:
coppieProf

MultiIndex([(    0,     0),
            (    1,     1),
            (    2,     2),
            (    3,     3),
            (    4,     4),
            (    5,     5),
            (    6,     6),
            (    7,     7),
            (    8,     8),
            (    9,     9),
            ...
            (27086, 27086),
            (27087, 27087),
            (27088, 27088),
            (27089, 27089),
            (27090, 27090),
            (27091, 27091),
            (27092, 27092),
            (27093, 27093),
            (27094, 27094),
            (27095, 27095)],
           length=28709)

In [88]:
#splitto le coppie 
train_coppieProf = coppieProf[:14354]
test_coppieProf = coppieProf[14354:]

In [96]:
compare = recordlinkage.Compare()

compare.string('name', 'name', method='jarowinkler', label="name", threshold=0.80)
compare.string('city', 'city', method='levenshtein', label="city", threshold=0.80)
compare.string('sector', 'sector', method='levenshtein', label="sector", threshold=0.85)
compare.string('website', 'website', method='levenshtein', label="website", threshold=0.85)
compare.string('specialization', 'specialization', method='levenshtein', label="specialization", threshold=0.90)
compare.string('languages', 'languages', method='levenshtein', label="languages", threshold=0.90)
compare.string('tel', 'tel', method='levenshtein', label="tel", threshold=0.90)


training_features = compare.compute(train_coppieProf, datiProf, datiProf)
training_features['score'] = training_features.loc[:, 'name':'city'].sum(axis=1)

In [98]:
training_matches = training_features[training_features.sum(axis=1) > 4].reset_index()
training_matches['score'] = training_matches.loc[:, 'name':'city'].sum(axis=1)
toDrop = ['name', 'city', 'website', 'specialization', 'languages', 'sector','tel','score']
training_matches = training_matches.drop(toDrop, axis=1)
training_matches.tail()

,level_0,level_1
14322,14295,14295
14323,14296,14296
14324,14297,14297
14325,14298,14298
14326,14299,14299


In [99]:
training_matches = pd.MultiIndex.from_frame(training_matches) #matches conversion

In [100]:
classifier = recordlinkage.KMeansClassifier()
classifier.fit(training_features)

In [101]:
compare = recordlinkage.Compare()

compare.string('name', 'name', method='jarowinkler', label="name", threshold=0.80)
compare.string('city', 'city', method='levenshtein', label="city", threshold=0.80)
compare.string('sector', 'sector', method='levenshtein', label="sector", threshold=0.85)
compare.string('website', 'website', method='levenshtein', label="website", threshold=0.85)
compare.string('specialization', 'specialization', method='levenshtein', label="specialization", threshold=0.90)
compare.string('languages', 'languages', method='levenshtein', label="languages", threshold=0.90)
compare.string('tel', 'tel', method='levenshtein', label="tel", threshold=0.90)

test_features = compare.compute(test_coppieProf, datiProf, datiProf)
test_features['score'] = test_features.loc[:, 'name':'city'].sum(axis=1)

In [104]:
test_matches = test_features[test_features.sum(axis=1) > 2].reset_index()
test_matches['score'] = test_matches.loc[:, 'name':'city'].sum(axis=1)
toDrop = ['name', 'city', 'website', 'specialization', 'languages', 'sector','tel','score']
test_matches = test_matches.drop(toDrop, axis=1)
test_matches

,level_0,level_1
0,14300,14300
1,14301,14301
2,14302,14302
3,14303,14303
4,14304,14304
...,...,...
14350,27091,27091
14351,27092,27092
14352,27093,27093
14353,27094,27094


In [105]:
test_matches = pd.MultiIndex.from_frame(test_matches) #matches conversion
predictions = classifier.predict(test_features)

In [113]:
confusion_matrix = recordlinkage.confusion_matrix(test_matches, predictions, len(test_features))

recall = recordlinkage.recall(test_matches, predictions)
print('RECALL -->', recall)
precision = recordlinkage.precision(test_matches, predictions)
print('PRECISION -->', precision)
accuracy = recordlinkage.accuracy(test_matches, predictions, len(test_features))
print('ACCURACY -->', accuracy)

RECALL --> 0.7659665621734588
PRECISION --> 0.8213516431623432
ACCURACY --> 0.37596656217345875
